In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch_geometric
from torch_geometric.loader import DataLoader
import yaml

import Dataset
import Models
from utils import time_func

In [2]:
print(f"Torch version: {torch.__version__}")
print(f"Cuda available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Cuda device: {torch.cuda.get_device_name()}")
print(f"Cuda version: {torch.version.cuda}")
print(f"Torch geometric version: {torch_geometric.__version__}")

Torch version: 2.0.1+cu117
Cuda available: False
Cuda version: 11.7
Torch geometric version: 2.3.1


In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cpu')

In [4]:
PARAMS = yaml.safe_load(open('./config_pipeline.yaml'))

DATA_PATH = PARAMS['input_subset_pre_processed']
MESH_PATH = PARAMS['input_subset_grid']

ON_CLUSTER = False
PLOT_VERTICAL = True

HID_CHANNELS = PARAMS['hid_channels']
N_CLASSES = PARAMS['n_classes']
TRAIN_BATCH_SIZE = PARAMS['train_batch_size']
TEST_BATCH_SIZE = PARAMS['test_batch_size']
VALID_BATCH_SIZE = PARAMS['valid_batch_size']

if PARAMS['final_act'] == "sigmoid":
    FINAL_ACTIVATION = torch.sigmoid
elif PARAMS['final_act'] == "linear":
    FINAL_ACTIVATION = torch.nn.Linear(1, 1)

train_set = []
valid_set = []
test_set = []

graph_dict = {}
n_features = None # TODO FIX

#TIMESTAMP = time_func.start_time() # TODO test for performances

### Dataset instantiation

In [10]:
timestamp = time_func.start_time()
dataset = Dataset.PilotDataset(root=DATA_PATH, mesh_path=MESH_PATH)
time_func.stop_time(timestamp, "Dataset creation")

  ---  Dataset creation  ---  0.552 seconds.


In [6]:
import xarray as xr
mesh = xr.open_dataset(MESH_PATH)
mesh

<xarray.Dataset>
Dimensions:  (nod2: 8852366, nodes_subset: 757747, edges_subset: 2268763, n2: 2)
Dimensions without coordinates: nod2, nodes_subset, edges_subset, n2
Data variables:
    lon      (nod2) float64 ...
    lat      (nod2) float64 ...
    nodes    (nodes_subset) int32 ...
    edges    (edges_subset, n2) int32 ...

In [7]:
coo_links = [[], []]
tmp_edges = mesh.edges.values

for i in range(mesh.dims['edges_subset']):
    coo_links[0].append(tmp_edges[i][0])
    coo_links[1].append(tmp_edges[i][1])

tmp_edges = mesh.edges.values.T
tmp_edges


array([[1035320, 1035320, 1035320, ..., 7106911, 7106910, 7106912],
       [1035378, 1035324, 1035377, ..., 7106910, 7106912, 7106914]],
      dtype=int32)

In [8]:
coo_links = np.array(coo_links)
print(coo_links)
print(tmp_edges.T)

[[1035320 1035320 1035320 ... 7106911 7106910 7106912]
 [1035378 1035324 1035377 ... 7106910 7106912 7106914]]
[[1035320 1035378]
 [1035320 1035324]
 [1035320 1035377]
 ...
 [7106911 7106910]
 [7106910 7106912]
 [7106912 7106914]]


In [11]:
''''from torch_geometric.data import Data
data = Data(edge_index=coo_links)
data.is_undirected()
'''
dataset

PilotDataset(367)

In [15]:
dataset.get(2015, 1, 1).is_directed()

AttributeError: 'GlobalStorage' object has no attribute 'edge_index'